In [9]:
import sys
import csv
from scipy import ndimage as ndi
import pandas as pd
import numpy as np
import cloudvolume as cv
from cloudvolume.lib import Bbox
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.preprocessing.features import*

# Generating neighborhoods and saving to csv

Getting segment ids in use

In [10]:
#url = "s3://mouse-light-viz/precomputed_volumes/brain1"
url = "s3://mouse-light-viz/precomputed_volumes/test"
ngl_skel = NeuroglancerSession("s3://mouse-light-viz/precomputed_volumes/segment_test", mip=0)
#ngl_skel = NeuroglancerSession(url+"_segments", mip=1)
working_ids = []
for seg_id in range(300):
    try:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        working_ids.append(seg_id)
    except:
          pass

print (working_ids)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 13.97it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  6.65it/s]

[2, 7, 11, 12, 13, 14, 20, 24, 28, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 91, 97, 98, 99, 101, 103, 106, 107, 109, 111, 112, 115, 118, 120, 124, 126, 127, 129, 132, 133, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 214, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 240, 243, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 272, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 293, 294, 295, 296, 298]


Defining CSV file and printing useful segment id's

In [16]:
output_feats = 'neighborhood7x7_1.csv'
int_verts = pd.DataFrame()
flag = 'w';
ids = working_ids
print (working_ids)

[2, 7, 11, 12, 13, 14, 20, 24, 28, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 91, 97, 98, 99, 101, 103, 106, 107, 109, 111, 112, 115, 118, 120, 124, 126, 127, 129, 132, 133, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 214, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 240, 243, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 272, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 293, 294, 295, 296, 298]


Selecting few segments to generate neighborhoods.

In [19]:
ids = working_ids
sel_ids=[]
for i in range(4,len(ids),5):
    sel_ids.append(ids[i])
#sel_ids

7x7x7 neighborhoods generation and saving to csv files. (Takes a while to run). <br>
Only first 50 vertex points from every segment are used. Offset of (3,3,3) on either side gives 7x7x7 neighborhood.

In [18]:
df = pd.DataFrame()
with open(output_feats,flag) as f:
    for seg_id in sel_ids:
        print(seg_id)
        segment = ngl_skel.cv.skeleton.get(seg_id)
        print("segment_id", seg_id)
        for v_id,vertex in enumerate(segment.vertices):
            ngl = NeuroglancerSession(url,mip=0)
            try:
                #img, bounds, voxel = ngl.pull_voxel(seg_id,v_id,nx=0,ny =0,nz = 0)
                if (v_id<50):
                    voxel = ngl_skel._get_voxel(seg_id, v_id)
                    n_offset = np.array([3,3,3])
                    bounds = Bbox(np.subtract(voxel,n_offset),np.add(voxel,n_offset+1))
                    img = ngl.pull_bounds_img(bounds)
                    r = img.shape
                    feat = img.flatten().reshape(r[0]*r[1]*r[2],1)

                    offset = np.array([15,15,15])
                    v_off = voxel + offset
                    bounds_off = Bbox(np.subtract(v_off,n_offset),np.add(v_off,n_offset+1))
                    img_off = ngl.pull_bounds_img(bounds_off)
                    r_off = img.shape
                    feat_off = img_off.flatten().reshape(r_off[0]*r_off[1]*r_off[2],1)

                    df = df.append(
                        {
                            "Segment": int(seg_id),
                            "Vertex": int(v_id),
                            "Label": 1,
                            "Features": feat,
                        },
                        ignore_index=True,
                    )
                    
                    df = df.append(
                        {
                            "Segment": int(seg_id),
                            "Vertex": int(v_id),
                            "Label": 0,
                            "Features": feat_off,
                        },
                        ignore_index=True,
                    )
                    t0 = df['Features'][0]
                    t0 = t0.transpose()
                    t0 = np.append(t0,int(df['Label'][0]))
                    t1 = df['Features'][1]
                    t1 = t1.transpose()
                    t1 = np.append(t1,int(df['Label'][1]))
                    
                    writer = csv.writer(f)
                    writer.writerows([t0])
                    writer.writerows([t1])
                    #print(df)
                    df = pd.DataFrame()
                    
                else:
                    break
            except:
                print("gone")
                pass

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

13


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


segment_id 13


Downloading: 100%|██████████| 1/1 [00:00<00:00, 12.38it/s]

...


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

291
segment_id 291


Downloading: 100%|██████████| 1/1 [00:00<00:00,  8.63it/s]

298
segment_id 298



Downloading: 100%|██████████| 1/1 [00:00<00:00, 26.47it/s]
